In [ ]:
import datetime
import numpy as np
import os
import pandas as pd
import spektral
import tensorflow as tf
import tensorflow_addons as tfa

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.engine.input_layer import InputLayer
from keras.models import load_model
from keras.layers import Activation
from keras import backend as K
from scipy.sparse import csr_array, csr_matrix, load_npz

tf.keras.backend.set_floatx('float32')

In [ ]:
class GraphDataset(spektral.data.Dataset):
    def __init__(self, path, **kwargs):
        self.data_path = path
        super().__init__(**kwargs)

    def read(self):
        output = []
        for i in range(int(len(os.listdir(self.data_path))/2)):
            graph = np.load(self.data_path + "graph_{}.npz".format(i))
            adjacency = load_npz(self.data_path + "adjacency_{}.npz".format(i))
            output.append(spektral.data.graph.Graph(x=graph['x'],
                                                    a=adjacency,
                                                    y=graph['y']))
        return output

In [ ]:
train_data = GraphDataset("dataset/train/")
test_data = GraphDataset("dataset/test/")
val_data = GraphDataset("dataset/validation/")

In [ ]:
def get_generator(dataset):
    def data_generator():
        for graph in dataset:
            yield graph.x[0,:6], graph.y
    return data_generator

In [ ]:
train_ds = tf.data.Dataset.from_generator(get_generator(train_data), 
                                          output_signature=(tf.TensorSpec(shape=6, dtype=tf.float32),
                                                            tf.TensorSpec(shape=(), dtype=tf.float32)))
val_ds = tf.data.Dataset.from_generator(get_generator(val_data),
                                        output_signature=(tf.TensorSpec(shape=6, dtype=tf.float32),
                                                            tf.TensorSpec(shape=(), dtype=tf.float32)))
test_ds = tf.data.Dataset.from_generator(get_generator(test_data),
                                         output_signature=(tf.TensorSpec(shape=6, dtype=tf.float32),
                                                            tf.TensorSpec(shape=(), dtype=tf.float32)))

train_ds = train_ds.batch(64)
val_ds = val_ds.batch(64)
test_ds = test_ds.batch(64)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(12, activation="relu"),
    tf.keras.layers.Dense(12, activation="relu"),
    tf.keras.layers.Dense(6, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid"),
])

In [ ]:
model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.losses.MeanSquaredError(), tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
log_dir = "logs/baseline_rightencoding/"
log_dir_train = log_dir + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir_train, histogram_freq=1)
early_stopping = EarlyStopping(monitor='val_loss',
                               restore_best_weights=True, patience=50,
                               verbose=0, mode='min')

In [ ]:
model.fit(train_ds,
          epochs=50,
          validation_data=val_ds,
          callbacks=[tensorboard, early_stopping],
          verbose=1)

In [ ]:
model.summary()

In [ ]:
model.save_weights(log_dir + "model_weights")